<a href="https://colab.research.google.com/github/KT2001/Omdena_Algeria_Project_OneShotLearning/blob/main/Model_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Verification notebook

In this notebook we have tried to verrify the result of our notebook with the verification dataset. https://drive.google.com/file/d/1XuCdX4vgAJdWhjBkvqluB8UvI85Yq9pK/view?usp=sharing

In [ ]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

# visulizations
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False
import seaborn as sns
sns.set(style="ticks", context="talk")
plt.style.use("dark_background")
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import os
import random
import time
import cv2
import glob
from scipy import misc
import pathlib

In [ ]:
# creating a preprocessing function
def preprocess(path):
  # read an image from file path
  byte_img = tf.io.read_file(path)
  # Load the image
  img = tf.io.decode_jpeg(byte_img)

  # preprocess steps - resize image to be 105x105x3
  img = tf.image.resize(img, (105, 105))
  # scaling the image between 0, 1
  img = img/255.0

  return img

In [ ]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
# Reload model 
siamese_model = tf.keras.models.load_model('/content/drive/MyDrive/PlantVillage/siamesemodelV2.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
siamese_model.summary()

Model: "Siamese_Network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'validation_img[0]

In [ ]:
!unzip /content/drive/MyDrive/PlantVillage/Verification_images_zip.zip

In [ ]:
ver_image = os.path.join('/content/Verification_images')

In [ ]:
def renamer(path):
  i = 0
  for filename in os.listdir(path):
      my_dest ="image" + str(i) + ".jpg"
      my_source =path +'/'+ filename
      my_dest =path +'/'+ my_dest
      # rename() function will
      # rename all the files
      os.rename(my_source, my_dest)
      i += 1

In [ ]:
renamer(ver_image)

In [ ]:
def verify(model, detection_threshold, verification_threshold, IMAGE):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('/content/Verification_images')):
        input_img = preprocess(os.path.join(IMAGE))
        validation_img = preprocess(os.path.join('/content/Verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('/content/Verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [ ]:
results, verified = verify(siamese_model, 0.5, 0.5, '/content/image(3).JPG')

1/1 [==============================] - 0s 21ms/step


In [ ]:
print(verified)

True
